In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import matplotlib as plt
import time

from model import CustomResNet
from torchvision.models.resnet import BasicBlock, Bottleneck

from torchvision.datasets import CIFAR10, CIFAR100
from torchvision.transforms import ToTensor, Compose, RandomCrop, RandomHorizontalFlip, Normalize
from torch.utils.data import DataLoader, random_split
from torch.optim import SGD, lr_scheduler
from tqdm import tqdm

dataset_name = 'cifar100'
student_name = 'resnet38_baseline'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_class = 100
batch_size = 128
epoch = 180
gamma = 0.2
milestones = [60, 120, 150]

In [2]:
transform = Compose([
    ToTensor(),
    RandomCrop(size=[32, 32], padding=4),
    RandomHorizontalFlip(p=0.5),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

transform_test = Compose([
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_dataset = CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = CIFAR100(root='./data', train=False, transform=transform_test, download=True)

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# 데이터셋 확인
print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(test_dataset)}')

Files already downloaded and verified
Files already downloaded and verified
Train dataset size: 50000
Validation dataset size: 10000


In [3]:
model = CustomResNet(block=BasicBlock,
                   layers=[6, 6, 6],
                   num_classes=num_class).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters: {total_params}')
print(model)

Total number of parameters: 569972
CustomResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layers): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3),

In [4]:
optimizer = SGD(params=model.parameters(), lr=0.1, nesterov=True, momentum=0.9, weight_decay=0.0001)
scheduler = lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=milestones, gamma=gamma)
criterion = nn.CrossEntropyLoss()

In [5]:
history = dict(train_loss=[], test_acc=[], train_time=[], test_time=[])
for ep in range(epoch):
    # train step
    train_loss = 0.0
    model.train()
    s_time = time.time()
    for image, target in train_loader:
        image = image.to(device)
        target = f.one_hot(target, num_class).float().to(device)

        pred = model(image)
        loss = criterion(pred, target)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    e_time = time.time()
    history['train_loss'].append(train_loss/len(train_loader))
    history['train_time'].append(e_time - s_time)

    # test step
    test_acc = 0.0
    model.eval()
    s_time = time.time()
    for image, target in test_loader:
        image = image.to(device)
        target = f.one_hot(target, num_class).float().to(device)

        pred = model(image)
        test_acc += torch.sum(torch.argmax(pred, dim=1) == torch.argmax(target, dim=1)).item()
    e_time = time.time()
    history['test_acc'].append(test_acc/len(test_dataset))
    history['test_time'].append(e_time - s_time)
    print(f'epoch={ep:3d}, train_loss={train_loss/len(train_loader):.4f}, test_acc={test_acc/len(test_dataset):.3f}')

    checkpoint = dict(
        model=model.state_dict(),
        optimizer=optimizer.state_dict(),
        history=history,
        epoch=ep
    )
    torch.save(checkpoint, f'./result/{dataset_name}_{student_name}.pt')
    scheduler.step()

epoch=  0, train_loss=4.0690, test_acc=0.109
epoch=  1, train_loss=3.4937, test_acc=0.188
epoch=  2, train_loss=3.0126, test_acc=0.278
epoch=  3, train_loss=2.6186, test_acc=0.335
epoch=  4, train_loss=2.3225, test_acc=0.393
epoch=  5, train_loss=2.1022, test_acc=0.411
epoch=  6, train_loss=1.9345, test_acc=0.434
epoch=  7, train_loss=1.8037, test_acc=0.465
epoch=  8, train_loss=1.7150, test_acc=0.463
epoch=  9, train_loss=1.6302, test_acc=0.510
epoch= 10, train_loss=1.5611, test_acc=0.488
epoch= 11, train_loss=1.5005, test_acc=0.523
epoch= 12, train_loss=1.4488, test_acc=0.512
epoch= 13, train_loss=1.4027, test_acc=0.510
epoch= 14, train_loss=1.3705, test_acc=0.525
epoch= 15, train_loss=1.3294, test_acc=0.542
epoch= 16, train_loss=1.3020, test_acc=0.532
epoch= 17, train_loss=1.2647, test_acc=0.535
epoch= 18, train_loss=1.2358, test_acc=0.557
epoch= 19, train_loss=1.2121, test_acc=0.563
epoch= 20, train_loss=1.1869, test_acc=0.530
epoch= 21, train_loss=1.1708, test_acc=0.554
epoch= 22,